#  Install package

In [ ]:
# ! pip install pyvis
# ! pip install networkx
# ! pip install dash
# ! pip install altair
# ! pip install panel

# Import package

In [ ]:
import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
from pyvis.network import Network
import openai
from IPython.display import display, HTML, Code
from IPython.display import clear_output


# Define Solution class
Please run the following cell to define the functions

In [ ]:
# %load_ext autoreload
# %autoreload 2

import LLM_Geo_Constants as constants
import helper
# import LLM_Geo_kernel.Solution as Solution

from LLM_Geo_kernel import Solution

%load_ext autoreload
%autoreload 2

# Demonstration Cases

## Input task and data desciption

In [ ]:
# Case 1: population living near hazardous wastes

task_name ='Resident_at_risk_counting'

TASK = r"""1) Find out the total population that lives within a tract that contain hazardous waste facilities. The study area is North Carolina, US.
2) Generate a map to show the spatial distribution of population at the tract level and highlight the borders of tracts that have hazardous waste facilities.
"""

DATA_LOCATIONS = ["NC hazardous waste facility ESRI shape file location: https://github.com/gladcolor/LLM- Geo/raw/master/overlay_analysis/Hazardous_Waste_Sites.zip.",
                  "NC tract boundary shapefile location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_shp_37.zip. The tract id column is 'Tract'.",
                  "NC tract population CSV file location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv. The population is stored in 'TotalPopulation' column. The tract ID column is 'GEOID'."
                 ]




# Case 2: France_mobility_changes_2020
# task_name ='France_mobility_changes_2020'
# TASK = r'''
# 1) Show the 2020 human mobility monthly change rates of each administrative regions in a France map. Each month is a sub-map in a map matrix. The base of the change rate is January 2020. 
# 2) Draw a line chart to show the monthly change rate trends of all administrative regeions. Each region is a line, the x-axis is 2020 months, the legend is the region name.
# '''

# DATA_LOCATIONS = ["ESRI shapefile for France administrative regions:" + \
#                   "https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip. " + \
#                   "The 'GID_1' column is the administrative region code, 'NAME_1' column is the administrative region name.",
#                   "REST API url with parameters for human mobility data access:" + \
#                   "http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020." + \
#                   "The response is in CSV format. There are three columns in the response: " + \
#                   "place,date (format:2020-01-07), and intra_movement. 'place' column is the administractive region code of every country; codes for France administrative regions start with 'FRA'.",
#                  ]

  
# Case 3: COVID-19 prevalence trend
# task_name ='COVID_death_rate'
# TASK = r'''1) Draw a map to show the death rate (death/case) of COVID-19 among the countiguous US counties. Use the accumulated COVID-19 data of 2020.12.31 to compute the death rate. Use scheme ='quantiles' when plotting the map.  Set map projection to 'Conus Albers'. Set map size to 15*10 inches.  
# 2) Draw a scatter plot to show the correlation and trend line of the death rate with the senior resident rate, including the r-square and p-value. Set data point transparency to 50%, regression line as red.  Set figure size to 15*10 inches.  
# '''

# DATA_LOCATIONS = [
#                   r"COVID-19 data case in 2020 (county-level): https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv. This data is for daily accumulated COVID cases and deaths for each county in the US. There are 5 columns: date (format: 2021-02-01), county, state, fips, cases, deaths. ",   
#                   r"Contiguous US county boundary (ESRI shapefile): https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip. The county FIPS column is 'GEOID'. ",
#                   r"Census data (ACS2020): https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county.csv. THe needed columns are: 'FIPS', 'Total Population', 'Total Population: 65 to 74 Years', 'Total Population: 75 to 84 Years', 'Total Population: 85 Years and Over'. Drop rows with NaN cells after loading the used columns.",
#                  ]




# Case 4: Hospital_accessibility
# task_name ='Hospital_accessibility'

# TASK = r'''
# For each zipcode area in South Carolina (SC), calculate the distance from the centroid of the zipcode area to its nearest hospital, and then create a choropleth distance map (unit: km), also show the hospital.
# '''

# # TASK = r'Diplay the  zipcode area in South Carolina (SC) and SC hospital locations in a  map.'

# DATA_LOCATIONS = [
# r"SC zipcode boundary shapefile: https://github.com/GIBDUSC/test/raw/master/sc_zip_boundary.zip, the map projection is WGS1984.",
# r"SC hospitals:  https://github.com/gladcolor/spatial_data/raw/master/South_Carolina/SC_hospitals_with_emergency_room_cleaned.csv, location columns: longitude in 'POINT_X' column, latitude in 'POINT_Y' column.",          
# ]


  
save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)

# create graph
# model=r"gpt-3.5-turbo"
model=r"gpt-4"
solution = Solution(
                    task=TASK,
                    task_name=task_name,
                    save_dir=save_dir,
                    data_locations=DATA_LOCATIONS,
                    model=model,
                    )
print("Prompt to get solution graph:\n")
print(solution.graph_prompt)

## Get graph code from GPT API

In [ ]:
response_for_graph = solution.get_LLM_response_for_graph() 
solution.graph_response = response_for_graph
solution.save_solution()

clear_output(wait=True)
display(Code(solution.code_for_graph, language='python'))

## Execute code to generate the solution graphto generate the solution graph

In [ ]:
exec(solution.code_for_graph)
solution_graph = solution.load_graph_file()

# Show the graph
G = nx.read_graphml(solution.graph_file)  
nt = helper.show_graph(G)
html_name = os.path.join(os.getcwd(), solution.task_name + '.html')  
# HTML file should in the same directory. See:
# https://stackoverflow.com/questions/65564916/error-displaying-pyvis-html-inside-jupyter-lab-cell
nt.show(name=html_name)
# html_name

## Generate prompts and code for operations (functions)

In [ ]:
operations = solution.get_LLM_responses_for_operations()
solution.save_solution()

all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])

clear_output(wait=True)
display(Code(all_operation_code_str, language='python'))

In [ ]:
all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])
# print(all_operation_code_str)

## Generate prompts and code for assembly program

In [ ]:
assembly_LLM_response = solution.get_LLM_assembly_response()
solution.assembly_LLM_response = assembly_LLM_response
solution.save_solution()

clear_output(wait=True)
display(Code(solution.code_for_assembly, language='python'))

## Execute assembly code

In [ ]:
all_code = all_operation_code_str + '\n' + solution.code_for_assembly

# print(solution.code_for_assembly)
display(Code(all_code, language='python'))

# clear_output(wait=True)

all_code = solution.execute_complete_program(code=all_code, try_cnt=10)
# solution.direct_request_code = code
display(Code(all_code, language='python'))

In [ ]:
import geopandas as gpd
import pandas as pd
from io import StringIO, BytesIO
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import requests
from tqdm import tqdm

def load_france_admin_shp(france_admin_shp_url="https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip"):
    france_admin_gdf = gpd.read_file(france_admin_shp_url)
    return france_admin_gdf

def load_mobility_data(mobility_api_url="http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020"):
    try:
        response = requests.get(mobility_api_url)
        response.raise_for_status()
        data = pd.read_csv(io.BytesIO(response.content))
        return data
    except requests.exceptions.HTTPError as e:
        print("Error: " + str(e))
        return None

def process_mobility_data(mobility_data):
    mobility_data = mobility_data[mobility_data["place"].str.startswith("FRA")]
    mobility_data["date"] = pd.to_datetime(mobility_data["date"])
    mobility_data["month"] = mobility_data["date"].dt.month
    base_intra_movement = mobility_data[mobility_data["month"] == 1].groupby("place")["intra_movement"].mean()
    base_intra_movement = base_intra_movement.reset_index()
    base_intra_movement.columns = ["place", "base_intra_movement"]
    mobility_data = mobility_data.merge(base_intra_movement, on="place")
    mobility_data["change_rate"] = (mobility_data["intra_movement"] - mobility_data["base_intra_movement"]) / mobility_data["base_intra_movement"]
    return mobility_data

def join_mobility_and_admin(france_admin_gdf, france_mobility_data):
    france_admin_gdf["GID_1"] = france_admin_gdf["GID_1"].astype(str)
    france_mobility_data["place"] = france_mobility_data["place"].astype(str)
    france_admin_mobility_gdf = france_admin_gdf.merge(france_mobility_data, left_on="GID_1", right_on="place")
    return france_admin_mobility_gdf

def generate_map_matrix(france_admin_mobility_gdf):
    fig, axarr = plt.subplots(3, 4, figsize=(16, 9), sharex=True, sharey=True)
    for month in tqdm(range(1,13), desc="Generating map matrix"):
        ax = axarr[(month-1)//4, (month-1)%4]
        france_admin_mobility_gdf[france_admin_mobility_gdf["month"]==month].plot(column="change_rate", ax=ax, legend=True, cmap="coolwarm", vmin=-1, vmax=1, scheme="quantiles")
        ax.set_title("Month "+str(month), fontsize=12)
    fig.suptitle("2020 Human Mobility Monthly Change Rates by Administrative Region in France (%)", fontsize=16)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    return axarr

def generate_line_chart(france_mobility_data):
    line_chart = france_mobility_data.pivot_table(index="month", columns="place", values="change_rate").reset_index()
    ax = line_chart.plot(x="month", kind="line", fontsize=8, colormap="Spectral",title="2020 Monthly Human Mobility Change Rates by Administrative Region in France (%)")
    ax.set_xticks(range(1,13))
    ax.set_xticklabels(["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"])
    ax.set_xlabel("Month", fontsize=12)
    ax.set_ylabel("Change Rate (%)", fontsize=12)
    ax.legend(title="Administrative Regions", bbox_to_anchor=(1, 1), fontsize=8)
    return ax

# Execute the program
france_admin_gdf = load_france_admin_shp()
mobility_data = load_mobility_data()
france_mobility_data = process_mobility_data(mobility_data)
france_admin_mobility_gdf = join_mobility_and_admin(france_admin_gdf, france_mobility_data)

map_matrix = generate_map_matrix(france_admin_mobility_gdf)
plt.savefig("france_monthly_change_rates_map_matrix.png", dpi=300)
plt.show()

line_chart = generate_line_chart(france_mobility_data)
plt.savefig("france_monthly_change_rates_line_chart.png", dpi=300)
plt.show()

In [ ]:
stop